In [1]:
using NBInclude
@nbinclude("functions.ipynb")

All function imported


In [3]:
# experiments with MNIST dataset
k = 20
n1 = 100
x_dim = 400
trials = 20
tolerance = 1e-7; max_iter = 2000; out_toggle = 0


A = sample_fourier(n1, x_dim); A = A' #sub-sampled DCT matrix used in convex combination
B = randn(x_dim, n1)/sqrt(x_dim) #random matrix that is used in convex combination

epoch_to_load = 20 # Load the model and test set loader
_, _, decoder = load_model("trained_GNN/MNIST", epoch_to_load)
W1 = Flux.params(decoder)[1][1:n1,:]
_, s, _ = svd(W1)
W1 = W1 / s[1]


z = randn(k) # ground code vector

β_list = 0:.005:1
recovery_error_matrix = zeros(trials)
recon_error_matrix = zeros(trials)

α_list = []
F = dct(diagm(ones(x_dim)),2)
for β in β_list
    push!(α_list, subspace_incoherence(F, β*A + (1-β) * B) )
end

Loading model...Done


In [8]:
α_list_ideal = LinRange(α_list[1],α_list[end], 11)
index_list = []
for α_ideal in α_list_ideal
    temp = 0
    index = 0
    for i in 1:length(α_list)
        if abs(temp - α_ideal) > abs(α_list[i] - α_ideal)
            temp = α_list[i]
            index = i
        end
    end
    push!(index_list, index)
end
α_list[index_list]

11-element Vector{Any}:
 0.5947989534459752
 0.6341617899869989
 0.6747168092599839
 0.7159869185472328
 0.7565331389758632
 0.7987293623472298
 0.8369393628115629
 0.8797661705424289
 0.919277030979835
 0.9602388166015151
 1.0000000000000004

In [ ]:
opt = Flux.Optimise.ADAM(.1)
m_list = 15:5:150

phase_matrix_recovery = zeros(trials, length(m_list), length(β_list))
phase_matrix_recon = zeros(trials, length(m_list), length(β_list))

for i in 1:length(m_list)
    m = m_list[i]
    recovery_error_matrix = zeros(trials)
    recon_error_matrix = zeros(trials)
    for β in β_list
        recovery_error_list = []
        recon_error_list = []
        G = Chain(
            Dense(20, n1, relu, bias = false; init =(out,in) -> W1),
            Dense(n1, x_dim, I, bias = false; init =(out,in) -> β*A + (1-β) * B)
        )    #setup the generative network 

        for trials in 1:trials
            F_sub = sample_fourier(m, x_dim)  # subsampling DCT for measurement matrix
            F_sub = F_sub * sqrt(x_dim)/sqrt(m) #normalization          
            y = F_sub * G(z) # measurement vector using subsampled DCT matrix
    
            z_est = randn(k) #initialization for algorithm
            z_est = estimated_code(opt, G, y, F_sub, z_est; max_iter, tolerance, out_toggle)  # run a optimizer to solve the least squares problem

            push!(recovery_error_list, relative_error(z, z_est))    
            push!(recon_error_list, relative_error(G(z), G(z_est)))
        end

        recovery_error_matrix =  hcat(recovery_error_matrix, recovery_error_list)
        recon_error_matrix = hcat(recon_error_matrix, recon_error_list)
    end
    phase_matrix_recovery[:, i, :] = recovery_error_matrix[:, 2:end]
    phase_matrix_recon[:, i, :] = recon_error_matrix[:, 2:end]
end

In [ ]:
save("saved_data/Phase_plot_MNIST.jld", "phase_matrix_recovery", phase_matrix_recovery, "phase_matrix_recon", phase_matrix_recon, "α_list", α_list, "m_list", m_list, "trials", trials)

In [ ]:
M = zeros(size(phase_matrix_recovery[1,:,:]))
for trial in 1:trials
    M += (phase_matrix_recovery[trial,:,:] .< 1e-5)
end

M = zeros(size(phase_matrix_recon[1,:,:]))
for trial in 1:trials
    M += (phase_matrix_recovery[trial,:,:] .< 1e-5)
end

p1 = plot(colorview(Gray, reverse(M/20, dims = 1)), bg =:black, title = "recovery of code z₀", framestyle = :box)
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 1:6:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 1:6:length(m_list)])

# m_pred = 150*α_list.^2
# ff(x) = -27/135 * x +(1 + 27*150/135)
# plot!(1:length(α_list), ff.(m_pred), label =:false, linewidth = 2 )

# m_pred = 150*α_list.^3
# plot!(1:length(α_list), ff.(m_pred), label =:false, linewidth = 2 )

p2 = plot(colorview(Gray, reverse(M/20, dims = 1)), bg =:black, title = "recovery of signal G(z₀)",framestyle = :box)
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 1:6:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 1:6:length(m_list)])

plot(p1, p2, layout = 2, xlabel = "incoherence upper bound", ylabel = "number of measurements")
savefig("figures/Phase_plot_MNIST_percentage.pdf")

In [ ]:


M = sum(phase_matrix_recovery,dims =1)[1,:,:]/20; M[M .>1] .= 1
p1 = plot(colorview(Gray, reverse(M,dims=1)), framestyle = :box, bg =:black, title = "recovery of code z₀")
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 1:6:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 1:6:length(m_list)])

M = sum(phase_matrix_recon,dims =1)[1,:,:]/20; M[M .>1] .= 1
p2 = plot(colorview(Gray, reverse(M,dims=1)), framestyle = :box, bg =:black, title = "recovery of signal G(z₀)")
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 1:6:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 1:6:length(m_list)])

plot(p1, p2, layout = 2,  bg =:white)
savefig("figures/Phase_plot_MNIST_error.pdf")



In [40]:
A = randn(10,5)
F = qr(A, ColumnNorm())
F.Q


10×10 LinearAlgebra.QRPackedQ{Float64, Matrix{Float64}}:
 -0.0134974  -0.571548    0.0484511  …  -0.15539     0.0912901   0.33149
  0.459321   -0.114382   -0.462345      -0.270878   -0.147602    0.218353
  0.0175865  -0.17283    -0.121099       0.287659    0.453238   -0.306456
  0.516655   -0.158915    0.569113      -0.209897    0.277787   -0.195627
  0.277584    0.183464    0.101816       0.0903834  -0.616911    0.16387
 -0.627275   -0.0748755   0.223395   …  -0.0410973  -0.0866537   0.212373
  0.047378    0.360251   -0.405886       0.209334    0.177225   -0.208252
  0.180335   -0.284551    0.136329       0.842738   -0.160996    0.135108
  0.101278    0.58988     0.411329       0.0567307   0.197894    0.288694
  0.0777842   0.0845645  -0.180397       0.0925765   0.450234    0.702373

In [42]:
A = sparse(A)
F=qr(A)
Matrix(F.Q)

10×5 Matrix{Float64}:
 -0.552422    0.147239   -0.444561   -0.0837547   0.0848972
 -0.218972   -0.419655   -0.392229    0.432773   -0.129348
 -0.172131    0.0234606  -0.152083    0.109451   -0.739701
 -0.275715   -0.464938    0.193073   -0.555623    0.0526722
  0.113204   -0.312885    0.145646   -0.0906168  -0.498945
  0.0744108   0.627331   -0.125392   -0.234008   -0.226611
  0.339075   -0.130591   -0.264501    0.386238    0.255395
 -0.318923   -0.108528   -0.0814762  -0.143197    0.229957
  0.549643   -0.23687    -0.505435   -0.452579   -0.0540545
  0.0639505  -0.0954561   0.468098    0.217973   -0.0692931

In [34]:
qr(A, ColumnNorm())

QRPivoted{Float64, SparseMatrixCSC{Float64, Int64}}
Q factor:
10×10 LinearAlgebra.QRPackedQ{Float64, SparseMatrixCSC{Float64, Int64}}:
 -0.0861883   0.231239    -0.111709   …   0.551744   -0.388181    0.382036
  0.269857   -0.25501      0.418821       0.194508   -0.512881    0.0581088
  0.65146     0.351101    -0.199806      -0.331763   -0.162592    0.213799
 -0.24232     0.0615062    0.213434       0.0826129   0.388654   -0.0367233
 -0.379535    0.592851     0.245975      -0.261023   -0.0092495   0.420466
 -0.344163    0.00524294  -0.0676385  …   0.255113   -0.246755    0.0471776
 -0.0176224  -0.185994     0.465038      -0.0188625  -0.0800841  -0.0732275
  0.358423    0.368133     0.0728693      0.62757     0.420531   -0.169717
 -0.0211613  -0.438297    -0.457911       0.115119    0.238584    0.539981
  0.21192    -0.201802     0.479898      -0.0136148   0.328063    0.546673
R factor:
5×5 SparseMatrixCSC{Float64, Int64} with 15 stored entries:
 3.42372   1.31991   0.90457    0.639715 